In [1]:
# TensorBoard is a powerful tool for visualizing and debugging machine learning models, 
# particularly in TensorFlow and Keras. It allows us to monitor key metrics, such as loss and accuracy,
# track training progress in real-time, and analyze the performance of various model components 
# throughout the training process

# Let's start by importing the California Housing dataset from Scikit-Learn
from sklearn.datasets import fetch_california_housing  # Load the California housing dataset
from sklearn.model_selection import train_test_split    # Tool to split data into train/validation/test sets
from sklearn.preprocessing import StandardScaler        # Tool to scale features to standard normal distribution

# Load the dataset (features and target)
housing = fetch_california_housing()

# Split data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

# Further split training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

# Standardize the features: zero mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)      # Fit on training data and transform it
X_valid = scaler.transform(X_valid)          # Use the same transformation on validation data
X_test = scaler.transform(X_test)            # Use the same transformation on test data

In [2]:
# Import the os module to interact with the operating system, specifically to create directories and handle paths.
import os

# Define the root directory where the log files will be stored.
# 'os.curdir' represents the current working directory. 'my_logs' is the subdirectory where logs will be saved.
root_logdir = os.path.join(os.curdir, "my_logs")

# Define a function to generate a unique directory path for each run.
# This function creates a directory name based on the current date and time to ensure unique names for each experiment.
def get_run_logdir():
    # Import the time module to work with time-based information.
    import time
    
    # Get the current time and format it as a string (e.g., 'run_2025_04_22-12_34_56').
    # This helps in creating a unique run ID for every new experiment.
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    
    # Return the full path to the log directory for the current run. This will be under the root_logdir.
    return os.path.join(root_logdir, run_id)

# Call the get_run_logdir function to create the log directory path for the current run.
# This will create a new subdirectory under 'my_logs' with a timestamped name.
run_logdir = get_run_logdir()

In [4]:
from tensorflow import keras  # Import Keras from TensorFlow to build and train deep learning models

# Enable eager execution in TensorFlow
# Eager execution allows operations to be evaluated immediately as they are called within Python, 
# rather than being added to a computational graph to be run later. 
# This is required to use certain functionalities like the .numpy() method on tensors. 
# If eager execution is not enabled, TensorFlow throws an error when trying to access .numpy() on tensors.
# This line ensures that eager execution is explicitly turned on, 
# even if it was disabled in some previous part of the code or environment.
import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)

# Load an already trained Keras model from an HDF5 file named 'nonsequential_model.h5'
# This file should contain the model architecture, weights, and optimizer state
model = keras.models.load_model("nonsequential_model.h5")

# Recreate the optimizer
optimizer = keras.optimizers.SGD(learning_rate=0.1, clipnorm=1.0)

# Recompile the model with the new optimizer
model.compile(optimizer=optimizer, loss='mse')

# Create a TensorBoard callback instance
# This will log training metrics such as loss and accuracy to the run_logdir directory
# These logs can be visualized in TensorBoard for better insight into the model's training progress
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

# Train the loaded model on the training data for 30 epochs
# - `X_train`, `y_train`: training features and targets
# - `validation_data`: a tuple of validation features and targets to evaluate the model after each epoch
# - `callbacks`: TensorBoard callback is passed to log training metrics
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 14s 39ms/step - loss: 0.5196 - val_loss: 0.4562
Epoch 2/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - loss: 0.4411 - val_loss: 0.4593
Epoch 3/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - loss: 0.4649 - val_loss: 0.3973
Epoch 4/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 21s 37ms/step - loss: 0.3792 - val_loss: 0.4472
Epoch 5/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.3584 - val_loss: 0.3633
Epoch 6/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 15s 41ms/step - loss: 0.3715 - val_loss: 0.4291
Epoch 7/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 15s 41ms/step - loss: 0.3797 - val_loss: 0.3684
Epoch 8/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - loss: 0.3620 - val_loss: 0.4367
Epoch 9/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.3470 - val_loss: 0.3926
Epoch 10/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - loss: 0.5422 - val_loss: 0.3761
Epoch 11/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - loss: 0.3502 - val_loss: 0.3545
Epoch 12/30
363/363 ━━━━━━━━━━

In [6]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

In [8]:
# This code has not been written for running
# It's a code for logging scalars, histograms, images, audio and text into TensorBoard
import numpy as np

test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(test_logdir)
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step/10), step=step)
        data = (np.random.randn(100) + 2) * step/100 # Some random data
        tf.summary.histogram("my_hist", data, buckets=50, step=step)
        images = np.random.rand(2, 32, 32, 3) # Some random 32x32 RGB images
        tf.summary.image("my_images", images * step / 1000, step=step)
        texts = ["The step is " + str(step), "It's square is " + str(step**2)]
        tf.summary.text("my_text", texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step= step)